In [7]:
!pip install -q deeplake==4.2.11 langchain==0.3.26 langchain-openai==0.3.23 langchain-core==0.3.66 langchain-deeplake==0.1.0 langchain-community==0.3.26 openai==1.92.0 tiktoken==0.8.0 python-dotenv jedi==0.19.2

In [8]:
import os

# os.environ["OPENAI_API_KEY"]="OPENAI_API_KEY"
# os.environ["ACTIVELOOP_TOKEN"]="YOUR_ACTIVELOOP_TOKEN"

from google.colab import userdata
os.environ["OPENAI_API_KEY"]= userdata.get('OPENAI_API_KEY')
os.environ["ACTIVELOOP_TOKEN"]= userdata.get('ACTIVELOOP_TOKEN')

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize the chat model
chat = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# Create the chat prompt template using modern tuple-based approach
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that translates english to pirate."),
    ("human", "Hi"),
    ("ai", "Argh me mateys"),
    ("human", "{text}")
])

# Create and run the chain
chain = chat_prompt | chat | StrOutputParser()
result = chain.invoke({"text": "I love programming."})
print("Pirate Translation Result:")
print(result)

Pirate Translation Result:
Arrr, I be lovin' me some programming, aye!


In [11]:
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate

# Create our examples
examples = [
    {
        "query": "What's the weather like?",
        "answer": "It's raining cats and dogs, better bring an umbrella!"
    },
    {
        "query": "How old are you?",
        "answer": "Age is just a number, but I'm timeless."
    }
]

# Create an example template
example_template = """
User: {query}
AI: {answer}
"""

# Create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# Create the prefix and suffix
prefix = """The following are excerpts from conversations with an AI
assistant. The assistant is known for its humor and wit, providing
entertaining and amusing responses to users' questions. Here are some
examples:
"""

suffix = """
User: {query}
AI: """

# Create the few-shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

# Create and run the chain
chain = few_shot_prompt_template | chat | StrOutputParser()
result2 = chain.invoke({"query": "What's the secret to happiness?"})
print("Few-Shot Humor Result:")
print(result2)

Few-Shot Humor Result:
The secret to happiness? Easy—find the joy in the little things, like perfectly toasted bread or a good pun. And remember, if all else fails, chocolate is a pretty reliable backup plan!


In [12]:
from langchain_core.example_selectors import LengthBasedExampleSelector

# Define examples for antonyms
examples_antonyms = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
    {"word": "energetic", "antonym": "lethargic"},
    {"word": "sunny", "antonym": "gloomy"},
    {"word": "windy", "antonym": "calm"},
]

# Create example template
example_template_antonyms = """
Word: {word}
Antonym: {antonym}
"""

example_prompt_antonyms = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_template_antonyms
)

# Create the example selector
example_selector = LengthBasedExampleSelector(
    examples=examples_antonyms,
    example_prompt=example_prompt_antonyms,
    max_length=25,
)

# Create dynamic prompt
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt_antonyms,
    prefix="Give the antonym of every input",
    suffix="Word: {input}\nAntonym:",
    input_variables=["input"],
    example_separator="\n",
)

# Test with different inputs
print("Length-Based Example Selector Results:")
print("Short input:")
print(dynamic_prompt.format(input="big"))
print("\nLong input:")
print(dynamic_prompt.format(input="extremely sophisticated"))

Length-Based Example Selector Results:
Short input:
Give the antonym of every input

Word: happy
Antonym: sad


Word: tall
Antonym: short


Word: energetic
Antonym: lethargic


Word: sunny
Antonym: gloomy

Word: big
Antonym:

Long input:
Give the antonym of every input

Word: happy
Antonym: sad


Word: tall
Antonym: short


Word: energetic
Antonym: lethargic

Word: extremely sophisticated
Antonym:


In [13]:
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_deeplake.vectorstores import DeeplakeVectorStore
from langchain_openai import OpenAIEmbeddings

# PromptTemplate
example_prompt_temp = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# Temperature conversion examples
examples_temp = [
    {"input": "0°C", "output": "32°F"},
    {"input": "10°C", "output": "50°F"},
    {"input": "20°C", "output": "68°F"},
    {"input": "30°C", "output": "86°F"},
    {"input": "40°C", "output": "104°F"},
]

# Create embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Setup DeepLake dataset
my_activeloop_org_id = "anais"  # TODO: Replace with your org ID
my_activeloop_dataset_name = "langchain_course_fewshot_selector"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"


# Create SemanticSimilarityExampleSelector using DeeplakeVectorStore
example_selector_semantic = SemanticSimilarityExampleSelector.from_examples(
    examples_temp,
    embeddings,
    DeeplakeVectorStore,
    k=1,
    dataset_path=dataset_path,
    overwrite=True
)

# FewShotPromptTemplate
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector_semantic,
    example_prompt=example_prompt_temp,
    prefix="Convert the temperature from Celsius to Fahrenheit",
    suffix="Input: {temperature}\nOutput:",
    input_variables=["temperature"],
)

# Test the similar_prompt with different inputs
print("Test 1:")
temp_result1 = (similar_prompt | chat | StrOutputParser()).invoke({"temperature": "10°C"})
print(f"10°C → {temp_result1}")

print("\nTest 2:")
temp_result2 = (similar_prompt | chat | StrOutputParser()).invoke({"temperature": "30°C"})
print(f"30°C → {temp_result2}")

# Add a new example to the SemanticSimilarityExampleSelector
similar_prompt.example_selector.add_example({"input": "50°C", "output": "122°F"})
print("\nTest 3 (after adding new example):")
temp_result3 = (similar_prompt | chat | StrOutputParser()).invoke({"temperature": "40°C"})
print(f"40°C → {temp_result3}")

print("\n" + "="*50 )

# DeeplakeVectorStore directly for more control
advanced_vectorstore = DeeplakeVectorStore(
    dataset_path=f"hub://{my_activeloop_org_id}/advanced_examples",
    embedding_function=embeddings,
    overwrite=True
)


# Chain for temperature conversion
temp_chain = similar_prompt | chat | StrOutputParser()

print("COMPLETE TEMPERATURE CONVERSION EXAMPLE")
print("="*50)

# Test the complete chain
test_temps = ["25°C", "15°C", "35°C"]
for temp in test_temps:
    result = temp_chain.invoke({"temperature": temp})
    print(f"{temp} → {result}")
    print("-" * 20)

Test 1:
10°C → Output: 50°F

Test 2:
30°C → 86°F

Test 3 (after adding new example):
40°C → Output: 104°F

COMPLETE TEMPERATURE CONVERSION EXAMPLE
25°C → Input: 25°C  
Output: 77°F
--------------------
15°C → Input: 15°C  
Output: 59°F
--------------------
35°C → Input: 35°C  
Output: 95°F
--------------------
